In [1]:
from pyldl.algorithms import LDL_DA

In [2]:
from pyldl.applications import facial_emotion_recognition as fer
from pyldl.utils import LDLEarlyStopping

In [3]:
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import keras

In [4]:
sX, sD = fer.load_bu_3dfe('BU-3DFE/', size=(128, 128))

In [5]:
tX, tD = fer.load_jaffe('jaffe/', size=(128, 128))

In [6]:
sD = LDL_DA.reorder_D(sD, LDL_DA.ORDER_SBU_3DFE)

In [7]:
tX_temp, tX_test, tD_temp, tD_test = train_test_split(tX, tD, train_size=int(tD.shape[1]*12.5), random_state=0)
tX_train, tX_val, tD_train, tD_val = train_test_split(tX_temp, tD_temp, test_size=.2, random_state=0)

In [8]:
model = LDL_DA()

In [9]:
enc_inputs = keras.Input(shape=sX.shape[1:])
encodes = keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3))(enc_inputs)
pooling = keras.layers.GlobalAveragePooling2D()(encodes)
encoder = keras.Model(inputs=enc_inputs, outputs=pooling)

In [10]:
dec_inputs = keras.Input(shape=(pooling.shape[1],))
outputs = keras.layers.Dense(sD.shape[1], activation='softmax')(dec_inputs)
decoder = keras.Model(inputs=dec_inputs, outputs=outputs)

In [11]:
model.fit(sX, sD, tX_train, tD_train, batch_size=128, model={'encoder': encoder, 'decoder': decoder},
          margin=1., optimizer=Adam(1e-5), ft_optimizer=Adam(1e-5), verbose=1,
          X_val=tX_val, D_val=tD_val, callbacks=[LDLEarlyStopping(monitor='kl_divergence', patience=None)]);

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 17913s 18s/step - loss: 2.1444 - kl_divergence: 0.0439
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 326s 326ms/step - loss: 0.0268 - kl_divergence: 0.0269


In [12]:
scores = model.score(tX_test, tD_test, return_dict=True)
for (k, v) in scores.items():
    print(f"{k}: {v:.4f}")

chebyshev: 0.0809
clark: 0.2999
canberra: 0.6386
kl_divergence: 0.0340
cosine: 0.9679
intersection: 0.8923
